# Credit Risk Resampling Techniques

In [1]:
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

Read the CSV into DataFrame

In [5]:
file_path = Path(r'C:\Users\boss\Downloads\lending_data.csv')
df = pd.read_csv(file_path)
df.head()

loan_size  interest_rate homeowner  borrower_income  debt_to_income  \
0    10700.0          7.672       own            52800        0.431818   
1     8400.0          6.692       own            43600        0.311927   
2     9000.0          6.963      rent            46100        0.349241   
3    10700.0          7.664       own            52700        0.430740   
4    10800.0          7.698  mortgage            53000        0.433962   

   num_of_accounts  derogatory_marks  total_debt loan_status  
0                5                 1       22800    low_risk  
1                3                 0       13600    low_risk  
2                3                 0       16100    low_risk  
3                5                 1       22700    low_risk  
4                5                 1       23000    low_risk

In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Fitting and encoding the columns with the LabelEncoder
le = LabelEncoder()

# Encoding homeowner column

le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])

df.head()

loan_size  interest_rate  homeowner  borrower_income  debt_to_income  \
0    10700.0          7.672          1            52800        0.431818   
1     8400.0          6.692          1            43600        0.311927   
2     9000.0          6.963          2            46100        0.349241   
3    10700.0          7.664          1            52700        0.430740   
4    10800.0          7.698          0            53000        0.433962   

   num_of_accounts  derogatory_marks  total_debt loan_status  
0                5                 1       22800    low_risk  
1                3                 0       13600    low_risk  
2                3                 0       16100    low_risk  
3                5                 1       22700    low_risk  
4                5                 1       23000    low_risk

Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [9]:
X.describe()

loan_size  interest_rate     homeowner  borrower_income  \
count  77536.000000   77536.000000  77536.000000     77536.000000   
mean    9805.562577       7.292333      0.606144     49221.949804   
std     2093.223153       0.889495      0.667811      8371.635077   
min     5000.000000       5.250000      0.000000     30000.000000   
25%     8700.000000       6.825000      0.000000     44800.000000   
50%     9500.000000       7.172000      1.000000     48100.000000   
75%    10400.000000       7.528000      1.000000     51400.000000   
max    23800.000000      13.235000      2.000000    105200.000000   

       debt_to_income  num_of_accounts  derogatory_marks    total_debt  
count    77536.000000     77536.000000      77536.000000  77536.000000  
mean         0.377318         3.826610          0.392308  19221.949804  
std          0.081519         1.904426          0.582086   8371.635077  
min          0.000000         0.000000          0.000000      0.000000  
25%          0.330357         3.000000          0.000000  14800.000000  
50%          0.376299         4.000000          0.000000  18100.000000  
75%          0.416342         4.000000          1.000000  21400.000000  
max          0.714829        16.000000          3.000000  75200.000000

In [10]:
X.describe().T

count          mean          std       min           25%  \
loan_size         77536.0   9805.562577  2093.223153   5000.00   8700.000000   
interest_rate     77536.0      7.292333     0.889495      5.25      6.825000   
homeowner         77536.0      0.606144     0.667811      0.00      0.000000   
borrower_income   77536.0  49221.949804  8371.635077  30000.00  44800.000000   
debt_to_income    77536.0      0.377318     0.081519      0.00      0.330357   
num_of_accounts   77536.0      3.826610     1.904426      0.00      3.000000   
derogatory_marks  77536.0      0.392308     0.582086      0.00      0.000000   
total_debt        77536.0  19221.949804  8371.635077      0.00  14800.000000   

                           50%           75%            max  
loan_size          9500.000000  10400.000000   23800.000000  
interest_rate         7.172000      7.528000      13.235000  
homeowner             1.000000      1.000000       2.000000  
borrower_income   48100.000000  51400.000000  105200.000000  
debt_to_income        0.376299      0.416342       0.714829  
num_of_accounts       4.000000      4.000000      16.000000  
derogatory_marks      0.000000      1.000000       3.000000  
total_debt        18100.000000  21400.000000   75200.000000

In [13]:
y.describe()

count        77536
unique           2
top       low_risk
freq         75036
Name: loan_status, dtype: object

In [16]:
y.values.T

array(['low_risk', 'low_risk', 'low_risk', ..., 'high_risk', 'high_risk',
       'high_risk'], dtype=object)

In [17]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [18]:
X_train.shape

(58152, 8)

In [19]:
X_test.shape

(19384, 8)

In [22]:
y_train.shape


(58152,)

In [23]:
y_test.shape

(19384,)

Data Pre-Processing

Scale the training and testing data using the StandardScaler from sklearn. Remember that when scaling the data, you only scale the features data (X_train and X_testing).

In [25]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [26]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [27]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Simple Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
bas = balanced_accuracy_score(y_test, y_pred)
print(bas)

0.9543211898288821


In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [31]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

>View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

Naive Random Oversampling

In [32]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

random = RandomOverSampler(random_state=1)
X_resampled, y_resampled = random.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [36]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946414201183431

In [38]:

# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



SMOTE Oversampling

In [41]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [42]:

# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [43]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9948279972279972

Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

>View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Display the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [44]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [45]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1) #, max_iter=2000
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [46]:
# Calculate the balanced accuracy score
y_pred= model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.8041461911615757

In [47]:
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [ 7258, 11501]], dtype=int64)

In [48]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.61      0.15      0.78      0.63       625
   low_risk       1.00      0.61      1.00      0.76      0.78      0.59     18759

avg / total       0.97      0.63      0.98      0.74      0.78      0.59     19384



Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

>View the count of the target classes using Counter from the collections library.
Use the resampled data to train a logistic regression model.
Calculate the balanced accuracy score from sklearn.metrics.
Display the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [49]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 74080, 'low_risk': 74591})

In [52]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
print(y_pred)
bc = balanced_accuracy_score(y_test, y_pred)
print(bc)
y_pred = model.predict(X_test)
print(y_pred)
cf=confusion_matrix(y_test, y_pred)
print(cf)
print(classification_report_imbalanced(y_test, y_pred))

['low_risk' 'low_risk' 'high_risk' ... 'low_risk' 'low_risk' 'low_risk']
0.994748035609574
['low_risk' 'low_risk' 'high_risk' ... 'low_risk' 'low_risk' 'low_risk']
[[  622     3]
 [  107 18652]]
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

Which model had the best balanced accuracy score?

>Ans: Simple Logistic Regression balanced accuracy score0.9543211898288821 Naive Random Oversampling balanced accuracy score 0.9934649587814939 SMOTE Oversampling balanced accuracy score 0.9936781215845847 Undersampling balanced accuracy score 0.8099934699520943 Combination balanced accuracy score 0.9935715401830394 The best balanced accuracy score was SMOTE but it is notable that SMOTE, Naive and Combination all had scores of higher than the simple logistic regression.

Which model had the best recall score?

>Ans: Simple Logistic Regression recall score .99 Naive Random Oversampling balanced recallscore .99 SMOTE Oversampling balanced recall score .99 Undersampling balanced recall score .64 Combination balanced recall score .99 All of the models with the exception of undersampling have a recall score of .99

Which model had the best geometric mean score?

>Ans: Simple Logistic Regression geo score .95 Naive Random Oversampling geo score .99 SMOTE Oversampling geo score .99 Undersampling geo score .79 Combination geo score .99 Naive random, SMOTE and combination all have a geo score of .99